In [2]:
import torch
from diffusers import ControlNetModel
from diffusers import StableDiffusionControlNetPipeline

from text3d2video.generative_rendering.generative_rendering_pipeline import (
    GenerativeRenderingPipeline,
)

# load pipeline
device = torch.device("cuda")
dtype = torch.float16

sd_repo = "runwayml/stable-diffusion-v1-5"
controlnet_repo = "lllyasviel/control_v11f1p_sd15_depth"

controlnet = ControlNetModel.from_pretrained(controlnet_repo, torch_dtype=dtype).to(
    device
)

print("controlnet loaded")

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_repo, controlnet=controlnet, torch_dtype=dtype
).to(device)

controlnet loaded


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

: 